<a href="https://colab.research.google.com/github/faithfulness121/beginner-project/blob/main/webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imprting liberies
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


In [2]:
#Retrieves the HTML content from a given URL.
def get_html_content(url):
    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')


In [3]:
            #1
#store the page url into a variable
url = "https://en.wikipedia.org/wiki/Mental_health"

#send a GET request to the page
response = requests.get(url)
#check if the request was successful
if response.status_code == 200:
#Parse the HTML content using BeautifulSoup
  soup = BeautifulSoup(response.content, 'html.parser')
else:
  print("No access:", response.status_code)

print(soup)

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Mental health - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-fea

In [4]:
                #2
# function to Extract article title

# Try to extract the article title
try:
    # Attempt to find the <h1> tag with id 'firstHeading' which contains the article title
    title_tag = soup.find("h1", id="firstHeading")

    if title_tag:
        # Extract and store the text content of the title
        title = title_tag.text.strip()
    else:
        # If the tag wasn't found, use a fallback message
        title = "Title not found"

except Exception as e:
    # Catch any unexpected errors during the process
    print(f"Error while extracting title: {e}")
    title = "Error retrieving title"

# Print or return the title
print(title)

Mental health


In [5]:
                      #3
# Find the main content division on the Wikipedia page by its specific class name
content_div = soup.find('div', class_='mw-parser-output')

# Initialize a paragraph counter
paragraph_count = 0

# Loop through all the HTML tags inside content_div that are either <h2>, <h3>, or <p>
for tag in content_div.find_all(['h2', 'h3', 'p']):

    # Check if the tag is a heading tag (<h2> or <h3>)
    if tag.name in ['h2', 'h3']:
        # Within the heading tag, find the <span> that holds the actual headline text
        span = tag.find('span', class_='mw-headline')

        # If a headline span is found, print the heading level and the text content
        if span:
            print(f"HEADING ({tag.name}): {span.text}")

    # If the tag is a paragraph (<p>)
    elif tag.name == 'p':
        paragraph_count += 1  # Increment paragraph counter
        # Print the paragraph number and the first 80 characters of the paragraph
        print(f"PARAGRAPH {paragraph_count}: {tag.text.strip()[:80]}...")


PARAGRAPH 1: Mental health encompasses emotional, psychological, and social well-being, influ...
PARAGRAPH 2: From the perspectives of positive psychology or holism, mental health is thus no...
PARAGRAPH 3: Mental health, as defined by the Public Health Agency of Canada,[7] is an indivi...
PARAGRAPH 4: Mental health is associated with a number of lifestyle factors such as diet, exe...
PARAGRAPH 5: In the mid-19th century, William Sweetser was the first to coin the term mental ...
PARAGRAPH 6: In American history, mentally ill patients were thought to be religiously punish...
PARAGRAPH 7: Emil Kraepelin in 1896 developed the taxonomy of mental disorders which has domi...
PARAGRAPH 8: At the beginning of the 20th century, Clifford Beers founded "Mental Health Amer...
PARAGRAPH 9: The mental hygiene movement, similar to the social hygiene movement, had at time...
PARAGRAPH 10: When US government-run hospitals were accused of violating human rights, advocat...
PARAGRAPH 11: It has been pro

In [6]:
#4:  Write a function to collect every link that redirects to another Wikipedia page

# Define a function to extract internal Wikipedia links from the parsed HTML (soup)
def extract_internal_links(soup):
    base_url = "https://en.wikipedia.org"
    links = set()

    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if href.startswith('/wiki/') and ':' not in href:
            full_url = urljoin(base_url, href)
            links.add(full_url)

    return list(links)

# Call and display the result
internal_links = extract_internal_links(soup)

# Print the number of internal links
print(f"\nTotal internal links: {len(internal_links)}")

# Print the first 10 internal links as a preview
for i, link in enumerate(internal_links[:10], start=1):
    print(f"Link {i}: {link}")


Total internal links: 854
Link 1: https://en.wikipedia.org/wiki/Generalized_anxiety_disorder
Link 2: https://en.wikipedia.org/wiki/Dissociative_fugue
Link 3: https://en.wikipedia.org/wiki/Student%27s_t-test
Link 4: https://en.wikipedia.org/wiki/Applied_behavior_analysis
Link 5: https://en.wikipedia.org/wiki/Behavioral_neuroscience
Link 6: https://en.wikipedia.org/wiki/Water_pollution
Link 7: https://en.wikipedia.org/wiki/Conduct_disorder
Link 8: https://en.wikipedia.org/wiki/South_Africa
Link 9: https://en.wikipedia.org/wiki/Epidemiology
Link 10: https://en.wikipedia.org/wiki/Social_cognitive_theory


In [7]:
#5: Wrap all the previous functions into a single function that takes as parameters a Wikipedia link

def extract_wikipedia_data(url, keyword_filter=None):
    """
    Extracts the title, structured content, and internal Wikipedia links from a given Wikipedia article URL.

    Parameters:
        url (str): The full Wikipedia page URL.
        keyword_filter (str, optional): A keyword to filter internal links (case-insensitive).

    Returns:
        dict: A dictionary with keys 'title', 'content', and 'internal_links'.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    base_url = "https://en.wikipedia.org"

    # Extract the article title
    title_tag = soup.find('h1', id='firstHeading')
    title = title_tag.text.strip() if title_tag else "No Title Found"

    # Extract structured article content
    content_div = soup.find('div', class_='mw-parser-output')
    content = {}
    current_heading = "Introduction"
    content[current_heading] = []

    for tag in content_div.find_all(['h2', 'h3', 'p']):
        if tag.name in ['h2', 'h3']:
            span = tag.find('span', class_='mw-headline')
            if span:
                current_heading = span.text.strip()
                content[current_heading] = []
        elif tag.name == 'p':
            paragraph = tag.get_text(strip=True)
            if paragraph:
                content[current_heading].append(paragraph)

    # Extract internal Wikipedia links
    links = set()
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if href.startswith('/wiki/') and ':' not in href:
            full_url = urljoin(base_url, href)
            if keyword_filter is None or keyword_filter.lower() in full_url.lower():
                links.add(full_url)

    return {
        "title": title,
        "content": content,
        "internal_links": list(links)
    }

wiki_url = "https://en.wikipedia.org/wiki/Mental_health"  # Example Wikipedia article

    # Call the function
extracted_data = extract_wikipedia_data(wiki_url, keyword_filter="health")

    # Print the title
print("Title:", extracted_data['title'])

    # Print the first two paragraphs of each section
print("\n--- Content ---")
for heading, paragraphs in extracted_data['content'].items():
  print(f"\n### {heading} ###")
for p in paragraphs[:2]:  # Only first 2 paragraphs per section
  print(f"- {p}")

    # Print number of internal links
  print(f"\nTotal internal links: {len(extracted_data['internal_links'])}")


Title: Mental health

--- Content ---

### Introduction ###
- Mental healthencompassesemotional,psychological, andsocialwell-being, influencingcognition,perception, andbehavior. Mental health plays a crucial role in an individual's daily life when managingstress, engaging with others, and contributing to life overall. According to theWorld Health Organization (WHO), it is a "state of well-being in which the individual realizes his or her abilities, cancopewith the normalstressesof life, canworkproductively and fruitfully, and can contribute to his or her community".[1]It likewise determines how an individual handlesstress,interpersonal relationships, anddecision-making.[2]Mental health includes subjective well-being, perceivedself-efficacy,autonomy, competence, intergenerational dependence, andself-actualizationof one'sintellectualand emotional potential, among others.[3]

Total internal links: 118
- From the perspectives ofpositive psychologyorholism, mental health is thus not merely 

In [8]:
# Step 6: Test on "Artificial intelligence" Wikipedia page
if __name__ == "__main__":
    test_url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
    result = extract_wikipedia_data(test_url)

    print(f"\nTitle: {result['title']}\n")

    print("=== Sample Sections and Text ===")
    for section, paragraphs in list(result['content'].items())[:2]:
        print(f"\n--- {section} ---")
        for paragraph in paragraphs[:2]:  # show first 2 paragraphs per section
            print(f"- {paragraph}")

    print(f"\nTotal internal links: {len(result['internal_links'])}")
    print("First 5 internal links:")
    for i, link in enumerate(result['internal_links'][:5], 1):
        print(f"{i}. {link}")




Title: Artificial intelligence

=== Sample Sections and Text ===

--- Introduction ---

Total internal links: 1771
First 5 internal links:
1. https://en.wikipedia.org/wiki/Technology_in_science_fiction
2. https://en.wikipedia.org/wiki/Policy_gradient_method
3. https://en.wikipedia.org/wiki/Journal_of_Consciousness_Studies
4. https://en.wikipedia.org/wiki/Philosophy_of_history
5. https://en.wikipedia.org/wiki/Determinism
